In [40]:
import numpy as np
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import torch
import torch.nn as nn
import torch.nn.functional as F

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from src.anchor.anchor_generator import (gen_base_anchors, get_anchors, 
                              grid_anchors, meshgrid)
from src.core.bbox.assigners import AssignResult
from src.datasets.loader.build_loader import build_dataloader
from src.models.builder import build_backbone, build_neck, build_head
from mmcv.runner import obj_from_dict
from mmcv.utils.config import Config
from src.core import multi_apply, weighted_smoothl1, weighted_sigmoid_focal_loss
from src.core.anchor import anchor_target_single, images_to_levels, unmap, anchor_inside_flags, expand_binary_labels

from src.core.bbox import assign_and_sample, build_assigner, PseudoSampler, bbox2delta

%store -r anchor_target_variable

In [41]:
anchor_target_variable

([tensor([[ -19.,   -7.,   26.,   14.],
          [ -25.,  -10.,   32.,   17.],
          [ -32.,  -14.,   39.,   21.],
          ...,
          [1163.,  342., 1524., 1065.],
          [1116.,  248., 1571., 1159.],
          [1057.,  129., 1630., 1278.]])],
 [tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0', dtype=torch.uint8)],
 [tensor([[ 748.0838,  304.4447,  980.2133,  747.1882],
          [ 707.9063,   46.1551, 1026.3268,  670.4366],
          [ 982.3378,  359.9517, 1055.0696,  458.0521],
          [1012.2678,  381.8004, 1073.9814,  452.5743]])],
 [None],
 [None],
 [{'ori_shape': (360, 640, 3),
   'img_shape': (750, 1333, 3),
   'pad_shape': (768, 1344, 3),
   'scale_factor': 2.0828125,
   'flip': False}],
 {'assigner': {'type': 'MaxIoUAssigner',
   'pos_iou_thr': 0.5,
   'neg_iou_thr': 0.4,
   'min_pos_iou': 0,
   'ignore_iof_thr': -1},
  'smoothl1_beta': 0.11,
  'gamma': 2.0,
  'alpha': 0.25,
  'allowed_border': -1,
  'pos_weight': -1,
  'debug': False})

In [42]:
def bbox_overlaps(bboxes1, bboxes2):
    """Calculate overlap between two set of bboxes.

    Args:
        bboxes1 (Tensor): shape (m, 4)
        bboxes2 (Tensor): shape (n, 4), if is_aligned is ``True``, then m and n
            must be equal.

    Returns:
        ious(Tensor): shape (m, n) 
    """

    rows = bboxes1.size(0)
    cols = bboxes2.size(0)

    if rows * cols == 0:
        return bboxes1.new(rows, 1) if is_aligned else bboxes1.new(rows, cols)

    lt = torch.max(bboxes1[:, None, :2], bboxes2[:, :2])  # [rows, cols, 2]
    rb = torch.min(bboxes1[:, None, 2:], bboxes2[:, 2:])  # [rows, cols, 2]

    wh = (rb - lt + 1).clamp(min=0)  # [rows, cols, 2]
    overlap = wh[:, :, 0] * wh[:, :, 1]
    area1 = (bboxes1[:, 2] - bboxes1[:, 0] + 1) * (
        bboxes1[:, 3] - bboxes1[:, 1] + 1)

    area2 = (bboxes2[:, 2] - bboxes2[:, 0] + 1) * (
        bboxes2[:, 3] - bboxes2[:, 1] + 1)
    ious = overlap / (area1[:, None] + area2 - overlap)

    return ious


In [43]:
def assign(bboxes, gt_bboxes, cfg_assigner, gt_bboxes_ignore=None, gt_labels=None):
    """Assign gt to bboxes.
    본 method는 각각의 anchor에 ground truth(이하 gt)를 할당하는 역할을 합니다.
    해당 작업이 완료되면 각각의 anchor는 -1, 0, 또는 어떠한 양수값을 갖게 됩니다.
    -1은 학습에 사용되지 않는 anchor를 뜻하며
    0은 negative sample을 뜻하고
    나머지 어떠한 양수값들은 할당된 gt bounding box의 index 번호를 뜻합니다.

    위 작업은 아래 정의된 assign_wrt_overlaps 함수에서 아래 순서대로 이뤄집니다.

    1. 모든 anchor box에 -1을 할당합니다.
    2. 모든 gt들과의 iou가 negative threshold값을 넘지 못하는 anchor box에 0을 할당합니다.
       다시 말해 어떠한 gt와도 유의미한 상관관계를 갖지 않는 anchor box를 negative sample로 사용한다는 말 입니다.
    3. gt와의 iou가 positive threshold를 넘는 anchor box에 대하여, 가장 가까운(iou가 가장 높은) gt의 index를 부여합니다.
    4. 각 gt에 대하여 가장 iou가 높은 anchor box에 해당 gt의 index값을 할당합니다. 
       3번 과정이 anchor box를 기준으로 이루어졌다면, 본 과정은 gt를 기준으로 이뤄집니다.
       만일 어떠한 gt를 기준으로 각 anchor들과의 iou가 모두 positive threshold보다 낮다면 해당 gt에 대해 학습할 positive sample이 없는 상황이 발생합니다.
       이럴 때에는 비록 iou가 충분치 않더라도 가장 가까운 anchor를 positive sample로 삼아 학습하게 됩니다.


    Args:
        bboxes (Tensor): 적절한 label을 할당해줘야 할 anchor box들, shape(n, 4).
        gt_bboxes (Tensor): Ground truth box들, shape (k, 4).
        gt_bboxes_ignore (Tensor, optional): Ground truth bboxes that are
            labelled as `ignored`, e.g., crowd boxes in COCO. 본 예제에서는 고려 대상이 아닙니다.
        gt_labels (Tensor, optional): ground truth bbox의 label, shape (k, ).

    Returns:
        :obj:`AssignResult`: 각 anchor에 적절한 label이 할당된 결과를 담는 객체를 생성하여 return.
    """

    if bboxes.shape[0] == 0 or gt_bboxes.shape[0] == 0:
        raise ValueError('No gt or bboxes')
    bboxes = bboxes[:, :4]
    overlaps = bbox_overlaps(gt_bboxes, bboxes)

    assign_result = assign_wrt_overlaps(overlaps, cfg_assigner, gt_labels)
    return assign_result


def assign_wrt_overlaps(overlaps, cfg_assigner, gt_labels=None):
    """Assign w.r.t. the overlaps of bboxes with gts.

    Args:
        overlaps (Tensor): Overlaps between k gt_bboxes and n bboxes,
            shape(k, n).
        gt_labels (Tensor, optional): Labels of k gt_bboxes, shape (k, ).

    Returns:
        :obj:`AssignResult`: The assign result.
    """
    if overlaps.numel() == 0:
        raise ValueError('No gt or proposals')

    num_gts, num_bboxes = overlaps.size(0), overlaps.size(1)

    # 1. assign -1 by default
    assigned_gt_inds = overlaps.new_full(
        (num_bboxes, ), -1, dtype=torch.long)

    # for each anchor, which gt best overlaps with it
    # for each anchor, the max iou of all gts
    max_overlaps, argmax_overlaps = overlaps.max(dim=0)
    # for each gt, which anchor best overlaps with it
    # for each gt, the max iou of all proposals
    gt_max_overlaps, gt_argmax_overlaps = overlaps.max(dim=1)

    # 2. assign negative: below
    assigned_gt_inds[(max_overlaps >= 0)
                     & (max_overlaps < cfg_assigner.neg_iou_thr)] = 0

    # 3. assign positive: above positive IoU threshold
    pos_inds = max_overlaps >= cfg_assigner.pos_iou_thr
    assigned_gt_inds[pos_inds] = argmax_overlaps[pos_inds] + 1

    # 4. assign fg: for each gt, proposals with highest IoU
    for i in range(num_gts):
        if gt_max_overlaps[i] >= cfg_assigner.min_pos_iou:
            max_iou_inds = overlaps[i, :] == gt_max_overlaps[i]
            assigned_gt_inds[max_iou_inds] = i + 1

    if gt_labels is not None:
        assigned_labels = assigned_gt_inds.new_zeros((num_bboxes, ))
        pos_inds = torch.nonzero(assigned_gt_inds > 0).squeeze()
        if pos_inds.numel() > 0:
            assigned_labels[pos_inds] = gt_labels[
                assigned_gt_inds[pos_inds] - 1]
    else:
        assigned_labels = None

    return AssignResult(
        num_gts, assigned_gt_inds, max_overlaps, labels=assigned_labels)

In [45]:
(anchor_list, valid_flag_list, gt_bboxes, gt_bboxes_ignore_list, gt_labels_list, img_metas, train_cfg) = anchor_target_variable

flat_anchors = anchor_list[0]
valid_flags = valid_flag_list[0]
gt_bboxes = gt_bboxes[0]
gt_bboxes_ignore = gt_bboxes_ignore_list[0]
gt_labels = gt_labels_list[0]
img_meta = img_metas[0]
cfg = train_cfg


inside_flags = anchor_inside_flags(flat_anchors, valid_flags,
                                   img_meta['img_shape'][:2],
                                   cfg.allowed_border)
# assign gt and sample anchors
anchors = flat_anchors[inside_flags, :]

assign_result = assign(anchors, gt_bboxes, cfg.assigner,
                                     gt_bboxes_ignore, gt_labels)
bbox_sampler = PseudoSampler()
sampling_result = bbox_sampler.sample(assign_result, anchors,
                                      gt_bboxes)

num_valid_anchors = anchors.shape[0]
bbox_targets = torch.zeros_like(anchors)
bbox_weights = torch.zeros_like(anchors)
labels = anchors.new_zeros(num_valid_anchors, dtype=torch.long)
label_weights = anchors.new_zeros(num_valid_anchors, dtype=torch.float)

pos_inds = sampling_result.pos_inds
neg_inds = sampling_result.neg_inds
if len(pos_inds) > 0:
    pos_bbox_targets = bbox2delta(sampling_result.pos_bboxes,
                                  sampling_result.pos_gt_bboxes)
    bbox_targets[pos_inds, :] = pos_bbox_targets
    bbox_weights[pos_inds, :] = 1.0
    if gt_labels is None:
        labels[pos_inds] = 1
    else:
        labels[pos_inds] = gt_labels[sampling_result.pos_assigned_gt_inds]
    if cfg.pos_weight <= 0:
        label_weights[pos_inds] = 1.0
    else:
        label_weights[pos_inds] = cfg.pos_weight
if len(neg_inds) > 0:
    label_weights[neg_inds] = 1.0

# map up to original set of anchors

num_total_anchors = flat_anchors.size(0)
labels = unmap(labels, num_total_anchors, inside_flags)
label_weights = unmap(label_weights, num_total_anchors, inside_flags)
bbox_targets = unmap(bbox_targets, num_total_anchors, inside_flags)
bbox_weights = unmap(bbox_weights, num_total_anchors, inside_flags)

print(labels, label_weights, bbox_targets, bbox_weights, pos_inds,
        neg_inds)

tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([1., 1., 1.,  ..., 1., 1., 1.]) tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]) tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]) tensor([ 76775,  78278,  78281,  78287,  78290,  78296,  78299,  79790,  79793,
         79799,  79802,  79808,  79811,  81302,  81311,  81320, 163115, 163866,
        163867, 163868, 163870, 163871, 163876, 163877, 163879, 163880, 163888,
        164614, 164615, 164622, 164623, 164624, 164625, 164626, 164627, 164628,
        164631, 164632, 164633, 164634, 164635, 164636, 164637, 164640, 164641,
        164643, 164644, 165371, 165378, 165379, 165380, 165382, 165383, 165384,
        165387, 165388, 165389, 165390, 165391, 165392, 165393, 165396, 165397,
        165399, 165400, 1661